In [ ]:
import cv2
import os
import csv
import imageio.v2 as imageio
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from PIL import Image
import base64
import io

def load_heic_image(image_path):
    print(f"Loading HEIC image: {image_path}", flush=True)
    image = imageio.imread(image_path)
    return image

# Define the folder containing your images and the output CSV file
image_folder = r'path_to_your_image_folder'
output_csv = r'path_to_your_csv_output.csv'

# Load sharpness data
sharpness_data = []
with open(output_csv, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header row
    for row in csvreader:
        sharpness_data.append((row[0], float(row[1])))

# Sort images by sharpness (ascending)
sharpness_data.sort(key=lambda x: x[1])

# Create interactive interface
current_index = 0
image_widget = widgets.Image()  
filename_widget = widgets.Label()
sharpness_widget = widgets.Label()

import base64

def show_image(index):
    global current_index
    current_index = index
    image_path = os.path.join(image_folder, sharpness_data[index][0])

    if image_path.lower().endswith('.heic'):
        img_np = load_heic_image(image_path)
        img = Image.fromarray(img_np)
    else:
        img = PILImage.open(image_path)

    # Convert to JPEG and then to base64
    buffered = io.BytesIO()
    img.save(buffered, format="JPEG")
    img_bytes = buffered.getvalue()  # Get raw bytes
    img_str = base64.b64encode(img_bytes).decode()

    # Display the image in the widget (corrected)
    image_widget.value = img_bytes  # Use raw bytes for the image widget
    filename_widget.value = f"Filename: {sharpness_data[index][0]}"
    sharpness_widget.value = f"Sharpness: {sharpness_data[index][1]}"

    print(f"Image {index + 1} out of {len(sharpness_data)}", flush=True)

def on_next(change):
    if current_index < len(sharpness_data) - 1:
        show_image(current_index + 1)

def on_prev(change):
    if current_index > 0:
        show_image(current_index - 1)

def on_delete(change):
    image_path = os.path.join(image_folder, sharpness_data[current_index][0])
    os.remove(image_path)
    del sharpness_data[current_index]
    print(f"Deleted: {image_path}", flush=True)
    if current_index < len(sharpness_data):
        show_image(current_index)
    else:
        show_image(current_index - 1)

next_button = widgets.Button(description="Next")
prev_button = widgets.Button(description="Previous")
delete_button = widgets.Button(description="Delete")

next_button.on_click(on_next)
prev_button.on_click(on_prev)
delete_button.on_click(on_delete)

display(prev_button, next_button, delete_button, image_widget, filename_widget, sharpness_widget)
show_image(0)  # Show the first image initially
